In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
import re 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
!pip install tensorflow-gpu
!pip install transformers[torch]
import transformers
import torch
from transformers import BertModel, BertConfig
import logging
from tqdm import notebook 

     |████████████████████████████████| 458.4 MB 7.4 kB/s eta 0:00:010   |█▉                              | 26.3 MB 3.9 MB/s eta 0:01:50     |███                             | 42.0 MB 6.3 MB/s eta 0:01:06     |██████▋                         | 94.7 MB 683 kB/s eta 0:08:52     |███████                         | 101.0 MB 5.7 MB/s eta 0:01:03     |████████▉                       | 126.8 MB 3.4 MB/s eta 0:01:39     |██████████████▌                 | 207.1 MB 6.3 MB/s eta 0:00:40     |███████████████▉                | 226.5 MB 5.3 MB/s eta 0:00:44     |█████████████████▉              | 255.2 MB 2.3 MB/s eta 0:01:30     |████████████████████████        | 344.6 MB 291 kB/s eta 0:06:30     |███████████████████████████▋    | 395.5 MB 6.3 MB/s eta 0:00:10
     |████████████████████████████████| 1.3 MB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 40.4 MB/s eta 0:00:01
     |████████████████████████████████|

     |████████████████████████████████| 2.8 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 895 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 197 kB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 271 kB/s eta 0:00:01
     |████████████████████████████████| 831.4 MB 3.5 kB/s eta 0:00:011  |▏                               | 4.2 MB 3.8 MB/s eta 0:03:37     |█▏                              | 28.9 MB 2.7 MB/s eta 0:05:00     |██▎                             | 58.6 MB 2.9 MB/s eta 0:04:32     |███▌                            | 92.1 MB 2.5 MB/s eta 0:05:02     |████                            | 105.8 MB 2.8 MB/s eta 0:04:22     |█████████▉                      | 254.8 MB 361 kB/s eta 0:26:37     |████████████                    | 310.0 MB 7.2 MB/s eta 0:01:13     |████████████▎                   | 318.3 MB 5.1 MB/s eta 0:01:41     |████████████▌                   | 325.5 MB 4.5 MB/s eta 0:01:53     |████████████▋                   | 327.5 

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

## Подготовка

Загрузим данные:

In [2]:
try:
    
    df = pd.read_csv('/content/drive/MyDrive/Тимур/DS/DS Part 4/Обработка текстов/Проект/toxic_comments.csv')
    
except:
    
    df = pd.read_csv('/datasets/toxic_comments.csv')
    
pd.options.display.max_columns = None


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Напишем функцию для просмотра данных:

In [4]:
def intro(x):
    """
    принимает: x - датасет
       выдаёт: общую информацию, первые пять строк и последние пять строк таблицы, 
               описание данных и количество дубликатов.
             
     описание: Функция дает первичную информацию о данных в датасете.
    """
    print('\033[1m' + 'Общая информация:')
    print ('\033[0m')
    display(x.info())
    print('\033[1m' + 'Первые пять строк:')
    print ('\033[0m')
    display(x.head())
    print('\033[1m' + 'Последние пять строк:')
    print ('\033[0m')
    display(x.tail())
    print('\033[1m' + 'Дубликаты:')
    print ('\033[0m')
    display(x.duplicated().sum())  

Посмотрим на данные:

In [5]:
intro(df)

Общая информация:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

Первые пять строк:



,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Последние пять строк:



,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


Дубликаты:



0

Проверим сколько у нас негативных и позитивных каментариев:

In [6]:
df['toxic'].value_counts(normalize = 1)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Токсичных коментариев значительно меньше.Всего 10 % от общего количества.

Переведём твиты в нужный формат:

In [7]:
corpus = df['text'].str.lower().values.astype('U')

Пишим функцию для лематизации и очистки текста:

In [8]:
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    lemm_list = lemmatizer.lemmatize(text)
    lemm_text = "".join(lemm_list)
    text_clear = re.sub(r'[^a-zA-Z]', ' ',lemm_text) 
    return " ".join(text_clear.split())
    

Применим её  тексту:

In [10]:
df['lem_clear_text'] = [lemmatize(corpus[i]) for i in range(len(corpus))]

Приведем строки к нижнему регистру:

Проверим:

In [11]:
df.head()

,text,toxic,lem_clear_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [ ]:
%%time
cross_val = ShuffleSplit(n_splits=5, test_size=0.25, random_state=12345)
catboost = CatBoostClassifier(loss_function='Logloss', class_weights =[0.4,1.1])
parameters_cat = {'depth':[5,10], 'learning_rate':np.arange(0.01,0.05,0.1),
                 'iterations': range(1000,2000,200)}
catboost_grid = catboost.grid_search(parameters_cat,
            Pool(features_train, target_train),
            cv=cross_val,
            verbose=True,
            plot=False)

Подготовим признаки и целевой признак:

In [12]:
target = df['toxic']
features = df['lem_clear_text']

Разделим данные на обучающую и тестовую выборки:

In [13]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, shuffle=False, random_state=180495)

Избавимся от стоп слов:

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('russian'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
tf_idf = count_tf_idf.fit_transform(features_train ) 

print("Размер матрицы:", tf_idf.shape)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Размер матрицы: (111699, 137276)


In [15]:
print('Размер обучающей выборки:', features_train.shape)
print('Разр тестовой выборки:', features_test.shape)

Размер обучающей выборки: (111699,)
Разр тестовой выборки: (47872,)


Токенизируем прзники:

In [16]:
features_train = count_tf_idf.fit_transform(features_train)
features_test = count_tf_idf.transform(features_test)

## Обучение

LGBM :

In [17]:
LGBM = LGBMClassifier(random_state = 180495,class_weight ='balanced',boosting_type = 'gbdt', objective = 'binary' )
 
LGBM .fit(features_train,target_train)
print('\033[1m', 'LGBM')
print('\033[0m')
prediction_t = LGBM.predict(features_train)
print(('F1 обучающая выборка: {:.2f}'.format(f1_score(target_train, prediction_t))))
prediction = LGBM.predict(features_test)
print(('F1 тестовая  выборка: {:.2f}'.format(f1_score(target_test, prediction))))
print('Параметры модели:', LGBM ) 

 LGBM

F1 обучающая выборка: 0.74
F1 тестовая  выборка: 0.70
Параметры модели: LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=-1, num_leaves=31,
               objective='binary', random_state=180495, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)


Выбором параметров в гридсерч думаю можно получить результат лучше, но не с моей машиной.

Логистическая регрессия:

In [18]:
params_LR= {'C': np.linspace(10, 20, num = 11, endpoint = True),
       'max_iter': [1000]}
LR = LogisticRegression(class_weight ='balanced', random_state= 180495)
  
gLR = GridSearchCV(LR, params_LR, cv=5, scoring='f1', n_jobs=-1, verbose=2)
gLR.fit(features_train,target_train)
print('\033[1m', 'Логистическая регрессия')
print ('\033[0m')
print(('F1 обучающая выборка: {:.2f}'.format(gLR.best_score_)))
prediction = gLR.best_estimator_.predict(features_test)
print(('F1 тестовая  выборка: {:.2f}'.format(f1_score(target_test, prediction))))
print(('Параметры модели:', gLR.best_estimator_ ))    


Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed: 11.8min finished


 Логистическая регрессия

F1 обучающая выборка: 0.77
F1 тестовая  выборка: 0.78
('Параметры модели:', LogisticRegression(C=12.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=180495, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False))


Попробуем улучшить результат с помощью BERT:

Загрузим предобученый токенизатор:

In [19]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')

Зададим размер выборки:

In [20]:
sample_size = 1000

Возьмем случайные признаки и случайные ответы:

In [21]:
corpus = df['lem_clear_text'] 
r_state=0
corpus_sample = corpus.sample(sample_size, random_state=r_state).reset_index(drop=True)
target_sample = pd.Series(target).sample(sample_size, random_state=r_state).reset_index(drop=True)
logging.getLogger('transformers.tokenization_utils').setLevel(logging.ERROR)

Пишем функцию для токенизации:

In [22]:
def bert_pad(text, max_len=512, attention_mask=True):
    """Выполняет токенизацию с помощью BERT, дополняет короткие объекты нулями до max_len""" 
    tokened = tokenizer.encode(text, add_special_tokens=True, max_length=max_len)
    return np.array(tokened + [0] * (max_len - len(tokened)))

Представим корпус в виде токенов BERT:

In [23]:
berted = []
for i in notebook.tqdm(iterable=range(corpus_sample.shape[0])):
    berted.append(bert_pad(corpus_sample[i]))

berted = np.vstack(berted)
berted.shape

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


(1000, 512)

Создадим маску массива:

In [24]:
attention_mask = np.where(berted != 0, 1, 0)


In [25]:
batch_size = 50
model = BertModel.from_pretrained('bert-large-uncased') 
embeddings = []

for i in notebook.tqdm(range(berted.shape[0] // batch_size)):
    batch = torch.LongTensor(berted[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].numpy()) 

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


обозначим признаки и целевой признак:

In [27]:
features = np.concatenate(embeddings)
target  = target_sample 


Разделим на обучающую и тестовую выборку :

In [28]:
train_features, test_features, train_target, test_target = train_test_split(
features, target, test_size=200)

Обучим модель:

In [36]:
params_LR= {'C': np.linspace(10, 20, num = 11, endpoint = True),
       'max_iter': [1000]}
LR = LogisticRegression(class_weight ='balanced', random_state= 180495)
  
gLR = GridSearchCV(LR, params_LR, cv=5, scoring='f1', n_jobs=-1, verbose=2)
gLR.fit(train_features, train_target)
print('\033[1m', 'Логистическая регрессия')
print ('\033[0m')
print(('F1 обучающая выборка: {:.2f}'.format(gLR.best_score_)))
prediction = gLR.best_estimator_.predict(test_features)
print(('F1 тестовая  выборка: {:.2f}'.format(f1_score(test_target, prediction))))
print(('Параметры модели:', gLR.best_estimator_ ))     

Fitting 5 folds for each of 11 candidates, totalling 55 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done  55 out of  55 | elapsed:   19.6s finished


 Логистическая регрессия

F1 обучающая выборка: 0.61
F1 тестовая  выборка: 0.52
('Параметры модели:', LogisticRegression(C=17.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=180495, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False))


Что ж результат не удовлетворительный но это всё из за того что было выбрано всего 1000 твитов.

## Выводы

Лучший результат дала логистическая регрессия с векторизацие признаков TfidfVectorizer
результат удовлетворяющий условию F1 тестовая  выборка: 0.78
Улучшить модель с помощью токенизации BERT не удалось,что связано с малой выборкой из которой извлекались энбендинги и училась модель, но это  затребовало много ресурсов.